## Environment with Simultaneously Stepping Agents

A good and simple example for a multi-agent env, in which all agents always step simultaneously is the Rock-Paper-Scissors game, in which two agents have to play N moves altogether, each choosing between the actions “Rock”, “Paper”, or “Scissors”. After each move, the action choices are compared. Rock beats Scissors, Paper beats Rock, and Scissors beats Paper. The player winning the move receives a +1 reward, the losing player -1.

In [11]:
# !pip install ray
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import gymnasium as gym


In [12]:
class RockPaperScissors(MultiAgentEnv):
    """
    Two player environment for the famous rock paper scissors game.
    Both players always move simultaneously over a course of 10 timesteps in total.
    The winner of each timestep receives reward of +1, the losing player -1.0.

    The observation of each player is the last opponent action.
    """

    ROCK = 0
    PAPER = 1
    SCISSORS = 2
    LIZARD = 3
    SPOCK = 4

    PLAYER1 = "player1"
    PLAYER2 = "player2"

    WIN_MATRIX = {
        (ROCK, ROCK): (0, 0),
        (ROCK, PAPER): (-1, 1),
        (ROCK, SCISSORS): (1, -1),
        (PAPER, ROCK): (1, -1),
        (PAPER, PAPER): (0, 0),
        (PAPER, SCISSORS): (-1, 1),
        (SCISSORS, ROCK): (-1, 1),
        (SCISSORS, PAPER): (1, -1),
        (SCISSORS, SCISSORS): (0, 0),
    }

    def __init__(self, config=None):
        super().__init__()

        # agents doesnt changes in episodes
        self.agents = self.possible_agents = [self.PLAYER1, self.PLAYER2]

        # The observations are always the last taken actions.
        # Hence observation and action spaces are identical.
        self.observation_spaces = self.action_spaces = {
            self.PLAYER1: gym.spaces.Discrete(3),
            self.PLAYER2: gym.spaces.Discrete(3),
        }

        self.last_move = None
        self.num_moves = 0

    def reset(self, *, seed=None, options=None):
        self.num_moves = 0

        observations = {
            self.PLAYER1: 0,
            self.PLAYER2: 0,
        }
        infos = {}

        return observations, infos

    def step(self, action_dict):
        self.num_moves += 1

        move1 = action_dict[self.PLAYER1]
        move2 = action_dict[self.PLAYER2]

        observations = {self.PLAYER1: move2, self.PLAYER2: move1}

        # Compute rewards for each player based on the win-matrix.
        r1, r2 = self.WIN_MATRIX[move1, move2]
        rewards = {self.PLAYER1: r1, self.PLAYER2: r2}

        # Terminate and truncate the entire episode (for all agents) once 10 moves have been made.
        terminateds = truncateds = {"__all__": self.num_moves >= 10}

        infos = {}
        return observations, rewards, terminateds, truncateds, infos

In [13]:
# Register the environment with RLlib
from ray.tune.registry import register_env

def env_creator(config=None):
    return RockPaperScissors(config)

register_env('rps_multiagent', env_creator)


In [14]:
from ray.rllib.algorithms.ppo import PPOConfig

# PPOConfig setup for RLlib
config = PPOConfig()
config = config.environment(env="rps_multiagent", env_config={})
config = config.multi_agent(
    policies={
        "player1": (None, gym.spaces.Discrete(3), gym.spaces.Discrete(3), {}),
        "player2": (None, gym.spaces.Discrete(3), gym.spaces.Discrete(3), {}),
    },
    policy_mapping_fn=lambda agent_id, *args, **kwargs: agent_id,
)
config.num_env_runners = 1

config.api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)

In [15]:
# Build the algorithm

algo = config.build_algo()

TypeError: '<' not supported between instances of 'Version' and 'ValueError'

In [9]:
# Train the agents
results = []

for i in range(20):
    result = algo.train()
    learner_stats = result['info']['learner']
    print(f"Iteration {i+1}:")
    for agent, stats in learner_stats.items():
        ls = stats['learner_stats']
        print(
            f"  {agent}: total_loss={ls['total_loss']:.3f}, "
            f"policy_loss={ls['policy_loss']:.3f}, "
            f"vf_loss={ls['vf_loss']:.3f}, "
            f"entropy={ls['entropy']:.3f}, "
            f"kl={ls['kl']:.3f}"
        )
    print(f"  env_steps_sampled={result['info']['num_env_steps_sampled']}")

NameError: name 'algo' is not defined

In [10]:
# Play a game using trained policies
import numpy as np

env = RockPaperScissors()
obs, _ = env.reset()
print("Initial observations: ", obs)

total_rewards = {env.PLAYER1: 0, env.PLAYER2: 0}
for step in range(10):
    actions = {}
    for agent_id in obs:
        # Convert scalar obs to one-hot for the policy
        obs_onehot = np.eye(3)[obs[agent_id]]
        policy = algo.get_policy(agent_id)
        action = policy.compute_single_action(obs_onehot)
        actions[agent_id] = int(action[0])

    # actions dict contains integer actions, which is correct for env.step
    obs, rewards, terminateds, truncateds, infos = env.step(actions)
    print(f"Step {step+1}: Actions: {actions}, Rewards: {rewards}")

    for agent_id in rewards:
        total_rewards[agent_id] += rewards[agent_id]

    if terminateds.get("__all__", False):
        break

print(f"Total rewards: {total_rewards}")

Initial observations:  {'player1': 0, 'player2': 0}


NameError: name 'algo' is not defined